# Process Data

Process event data for application

In [7]:
import json
import pprint
%pprint

Pretty printing has been turned ON


In [2]:
with open('./all_data.json') as f:
    all_data = json.load(f)

In [3]:
with open('./labels_with_events.json') as f:
    event_labels = json.load(f)

## Get Event Data

In [27]:
import xmltodict

In [74]:
with open('./events.xml') as f:
    dict_topics = xmltodict.parse(f.read())
    json_output = json.dumps(dict_topics)
    topics = json.loads(json_output)['topics']['top']
    for topic in topics:
        topic['narr'] = ' '.join(topic['narr'].replace('\n', '').split())

In [75]:
topic_names = [topic['dataset'] for topic in topics]

In [48]:
fixed_event_labels = [event for event in event_labels if event['eventid'] in topic_names]

In [53]:
fixed_event_labels[0]['tweets'][:1]

[{'postID': '390373897704448000',
  'timestamp': '8 Apr 2019 15:21:04 GMT',
  'categories': ['Hashtags', 'Sentiment', 'Irrelevant'],
  'indicatorTerms': [],
  'priority': 'Low',
  'text': 'RT @QueenPetraK: Prayers for Bohol and Cebu and other parts of Visayas. Keep safe everyone.  #PrayForVisayas God bless the Philippines!'}]

## Combine with All Text Data

In [59]:
ids = []
all_labels = []
for event in fixed_event_labels:
    for tweet in event['tweets']:
        all_labels.append(tweet)
        ids.append(tweet['postID'])

In [60]:
print(len(ids))

12560


In [56]:
intersecting_data = []
for tweet in all_data:
    if tweet['allProperties']['id'] in ids:
        intersecting_data.append(tweet)

In [57]:
len(intersecting_data)

12700

In [61]:
final_data = []
for tweet in intersecting_data:
    post_id = tweet['allProperties']['id']
    id_index = ids.index(post_id)
    tweet['label'] = all_labels[id_index]
    final_data.append(tweet)

In [64]:
whitelisted_properties = ['user.verified', 'id', 'text', 'created_at', 'user.name', 'user.profile_image_url_https']

In [68]:
search_data = []

for tweet in final_data:
    data_point = {
        'datasetid': tweet['datasetid'],
        'topic': tweet['topic'],
        'label': tweet['label']
    }
    properties = {}
    for key in tweet['allProperties'].keys():
        if key in whitelisted_properties:
            properties[key] = tweet['allProperties'][key]
    data_point['properties'] = properties
    search_data.append(data_point)

In [69]:
search_data[0]

{'datasetid': 'trecis2018-test',
 'topic': 'albertaFloods2013',
 'label': {'postID': '351734622884855808',
  'timestamp': '11 Sep 2018 19:36:19 GMT',
  'categories': ['ThirdPartyObservation', 'Advice'],
  'indicatorTerms': [],
  'priority': 'Low'},
 'properties': {'user.verified': 'false',
  'id': '351734622884855808',
  'text': '#yycflood be sure when you are ready for re building you call companies that have a biz license http://t.co/1H6ezAq5vD',
  'user.profile_image_url_https': 'https://pbs.twimg.com/profile_images/515940503939203072/50Uopj8l_normal.jpeg',
  'created_at': 'Mon Jul 01 16:10:43 +0000 2013',
  'user.name': 'HStreetz'}}

## Write Data

In [70]:
with open('search_data.json', 'w') as f:
    json.dump(search_data, f)

In [77]:
with open('events.json', 'w') as f:
    json.dump(topics, f)